In [1]:
import subprocess
import shlex

import numpy as np
import polars as pl
import pandas as pd
import networkx as nx
import scipy.io

In [2]:
phenotype_names = pl.read_csv("data/pheno/pheno_sub_0.tsv", separator="\t", n_rows=0).drop(["#FID", "IID"]).columns

idx_to_name = {i + 1: name for i, name in enumerate(phenotype_names)}

sorted(idx_to_name.items())[:5]

[(1, 'q_46_0'), (2, 'q_47_0'), (3, 'q_48_0'), (4, 'q_49_0'), (5, 'q_50_0')]

In [3]:
rg_df = pl.read_parquet("data/gathered_rg.parquet")

filtered_rg_df = (
    rg_df
    .filter(
        pl.col("Coefficient").eq("rG") &
        pl.col("Estimate").ge(-1) &
        pl.col("Estimate").le(1) &
        pl.col("SE_OLS").le(2) & 
        pl.col("SE_Jackknife").le(2)
    )
)

phenotypes = sorted(set(filtered_rg_df.select("pheno1", "pheno2").to_numpy().ravel().tolist()))
pheno_to_idx = {p: i for i, p in enumerate(phenotypes)}

filtered_rg_df = (
    filtered_rg_df
    .with_columns(
        pl.col("pheno1", "pheno2").map_elements(pheno_to_idx.get).name.suffix("_idx")
    )
)

edges = filtered_rg_df.select("pheno1_idx", "pheno2_idx").unique().to_numpy().tolist()
nodes = [pheno_to_idx[x] for x in phenotypes]

G = nx.from_edgelist(edges)
A = nx.adjacency_matrix(G, nodelist=nodes)

print(G.number_of_nodes(), "nodes", G.number_of_edges(), "edges")

scipy.io.mmwrite("data/full_graph.mtx", A, field="pattern", symmetry="symmetric")

command = "PMC -f data/full_graph.mtx"
result = subprocess.run(shlex.split(command), capture_output=True)
max_clique_line = result.stdout.decode().splitlines()[-1].strip()
max_clique = [int(i) - 1 for i in max_clique_line.split(": ")[1].split(" ")]

print(len(max_clique), "nodes in the max clique")

for i, node1 in enumerate(max_clique):
    for node2 in max_clique[i+1:]:
        assert A.toarray()[node1, node2] == 1

767 nodes 81267 edges
73 nodes in the max clique


In [4]:
genetic_covariance_matrix = (
    filtered_rg_df
    .filter(pl.col("pheno1_idx").is_in(max_clique) & pl.col("pheno2_idx").is_in(max_clique))
    .sort("SE_OLS")
    .select("pheno1", "pheno2")
    .join(rg_df.filter(pl.col("Coefficient").str.contains("Vp")), on=["pheno1", "pheno2"])
    .with_columns(
        pheno1=pl.when(pl.col("Coefficient").eq("V(G)/Vp_tr2")).then("pheno2").otherwise("pheno1"),
        pheno2=pl.when(pl.col("Coefficient").eq("V(G)/Vp_tr1")).then("pheno1").otherwise("pheno2"),
    )
    .with_columns(
        pl.col("pheno1", "pheno2").cast(pl.Int32).map_elements(idx_to_name.get).name.suffix("_name")
    )
    .select("pheno1_name", "pheno2_name", "Estimate")
    .unique()
    .pipe(
        lambda df: pl.concat([
            df,
            (
                df
                .filter(pl.col("pheno1_name").ne(pl.col("pheno2_name")))
                .select(pheno1_name="pheno2_name", pheno2_name="pheno1_name", Estimate="Estimate")
            )
        ])
    )
    .pivot(index="pheno1_name", columns="pheno2_name", values="Estimate")
    .to_pandas()
    .set_index("pheno1_name")
    .rename_axis(index="pheno1")
    .pipe(lambda df: df.loc[sorted(df.index), sorted(df.index)])
)

print(genetic_covariance_matrix.shape)

print("Min:", np.diag(genetic_covariance_matrix).min(), "Max:", np.diag(genetic_covariance_matrix).max())

genetic_covariance_matrix.iloc[:3, :3]

(73, 73)
Min: 0.0191717 Max: 0.101884


,b_A07,b_A38,b_B01
pheno1,,,
b_A07,0.042155,0.003387,-0.005208
b_A38,0.003387,0.023780,0.020762
b_B01,-0.005208,0.020762,0.042270


In [5]:
genetic_covariance_matrix.pipe(lambda df: df.loc[sorted(df.index), sorted(df.index)]).to_csv("data/gcov/gcov_sub_0.tsv", sep="\t")